## Compare the repeats between two scirpts

Ashley(AS): https://github.com/desihub/LSS/blob/main/scripts/get_repeat_redshifts.py

repeat_dir = '/global/cfs/cdirs/desi/survey/catalogs/DA2/LSS/loa-v1' 

local_saved = '/pscratch/sd/s/shengyu/repeats/DA2/loa-v1'

Anand(AN): https://github.com/desihub/LSS/blob/main/scripts/desi_main_repeats.py

repeat_dir = '/global/cfs/cdirs/desi/spectro/redux/main' 

local_saved = '/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1'

1. BGS bright, or + BGS faint?, double check the numbers
2. how to do the z selection for the repetas sample, which is the ture rate for catastrophics
3. QSO catas rate inconsistency between two scripts, TILEID issues?

In [21]:
import os, sys
import random
import fitsio
import numpy as np
from astropy.table import Table,join,unique,vstack
import matplotlib.pyplot as plt
from matplotlib import ticker
import random

sys.path.append('/global/homes/s/shengyu/project_rc/main/Y3/')
from helper import REDSHIFT_VSMEAR, REDSHIFT_LSS_VSMEAR, REDSHIFT_CUBICBOX, EDGES, COLOR_TRACERS, GET_RECON_BIAS
from Y3_redshift_systematics import vsmear, vsmear_modelling
%load_ext autoreload
%autoreload 2

Z_SMEAR = REDSHIFT_VSMEAR.copy()
Z_SMEAR['LRG'] = [(0.4, 1.1)]+Z_SMEAR['LRG']
Z_SMEAR['ELG'] = [(0.8, 1.6)]+Z_SMEAR['ELG']
Z_SMEAR['QSO'] = [(0.8, 2.1)]+Z_SMEAR['QSO']
c = 299792.458

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
def repeat_quantity(tracer, zmin, zmax, script='AS', dv_limit = 1000):
    if script == 'AN':
        REPEAT_DIR = f'/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1' 
        d = Table.read(REPEAT_DIR+f'/{tracer}repeats.fits', hdu=1) 
        sel      = np.full(len(d),True)
        # selz = ((zmin<d["Z_0"])&(d["Z_0"]<zmax))
        selz = ((zmin<d["Z_0"])&(d["Z_0"]<zmax))|((zmin<d["Z_1"])&(d["Z_1"]<zmax))
        d_zbin = d[sel & selz]     
        dv_zbin = (d_zbin['Z_1']-d_zbin['Z_0'])/(1+d_zbin['Z_0'])*c
    elif script == 'AS':
        REPEAT_DIR = f'/pscratch/sd/s/shengyu/repeats/DA2/loa-v1'
        d = Table.read(REPEAT_DIR+f'/{tracer}repeats.fits', hdu=1) 
        sel      = np.full(len(d),True)
        sel = np.isfinite(d['Z1']) & np.isfinite(d['Z2'])
        # selz = ((zmin<d["Z1"])&(d["Z1"]<zmax))
        # selz = ((zmin<d["Z1"])&(d["Z1"]<zmax))|((zmin<d["Z2"])&(d["Z2"]<zmax))
        # d_zbin = d[sel & selz]
        d_zbin = d[sel]
        dv_zbin = (d_zbin['Z2']-d_zbin['Z1'])/(1+d_zbin['Z1'])*c

    number = len(d_zbin) # number of repeats
    selcata = abs(dv_zbin) < dv_limit # select the catastrophics outliers
    logdv_zbin = np.log10(abs(dv_zbin)) 
    median_dv = 10**np.median(logdv_zbin) # median of the dv
    dv_smear = dv_zbin[selcata] # redshift uncertainty part
    RMS = np.sqrt(np.mean(dv_smear ** 2)) # RMS of the dv
    fc = (1-len(dv_smear)/len(dv_zbin))*100 # catastrophic rate in %
    qu = dict(number=number, median=median_dv, RMS=RMS, fc=fc)
    return qu

In [43]:
for tracer in ['BGS']:
# for tracer in ['QSO']:
    for i,(zmin, zmax) in enumerate(Z_SMEAR[tracer]):
        dv_limit = 3000 if tracer == 'QSO' else 1000
        i_suffix = i if i>0 else ''
        print(f'{tracer}{i_suffix}', (zmin, zmax), f'dv>{dv_limit} or dz>{dv_limit/c:.4f}' )
        for script in ['AN','AS']:
            qu = repeat_quantity(tracer, zmin, zmax, script=script, dv_limit=dv_limit)
            print(script,qu)

BGS (0.1, 0.4) dv>1000 or dz>0.0033
AN {'number': 244123, 'median': 14.408252769674963, 'RMS': 35.16199521313628, 'fc': 0.3756303175038833}
AS {'number': 363856, 'median': 13.367370240968773, 'RMS': 35.99937820719732, 'fc': 0.7404027967107818}


In [46]:
for tracer in ['QSO']:
# for tracer in ['QSO']:
    for i,(zmin, zmax) in enumerate(Z_SMEAR[tracer]):
        dv_limit = 3000 if tracer == 'QSO' else 1000
        i_suffix = i if i>0 else ''
        print(f'{tracer}{i_suffix}', (zmin, zmax), f'dv>{dv_limit} or dz>{dv_limit/c:.4f}' )
        for script in ['AN','AS']:
            qu = repeat_quantity(tracer, zmin, zmax, script=script, dv_limit=dv_limit)
            print(script,qu)

QSO (0.8, 2.1) dv>3000 or dz>0.0100
AN {'number': 279711, 'median': 95.68932864317246, 'RMS': 409.7945812870456, 'fc': 2.372448705985819}
AS {'number': 1359473, 'median': 70.77066019791768, 'RMS': 380.092656880503, 'fc': 0.7249868147436556}
QSO1 (0.8, 1.1) dv>3000 or dz>0.0100
AN {'number': 35664, 'median': 30.559673359265517, 'RMS': 194.8630878832419, 'fc': 3.900291610587703}
AS {'number': 1359473, 'median': 70.77066019791768, 'RMS': 380.092656880503, 'fc': 0.7249868147436556}
QSO2 (1.1, 1.4) dv>3000 or dz>0.0100
AN {'number': 54219, 'median': 52.716735316935726, 'RMS': 242.51617540978134, 'fc': 2.3866172374997663}
AS {'number': 1359473, 'median': 70.77066019791768, 'RMS': 380.092656880503, 'fc': 0.7249868147436556}
QSO3 (1.4, 1.7) dv>3000 or dz>0.0100
AN {'number': 69194, 'median': 139.9283536734217, 'RMS': 462.6280097864955, 'fc': 2.7169985836922317}
AS {'number': 1359473, 'median': 70.77066019791768, 'RMS': 380.092656880503, 'fc': 0.7249868147436556}
QSO4 (1.7, 2.1) dv>3000 or dz>0

In [29]:
qu = repeat_quantity('QSO', 0.8, 2.1, script='AS', dv_limit=10000)
print(qu)

{'number': 255111, 'median': 92.40748031676436, 'RMS': 484.27009190164233, 'fc': 1.0771781695026905}


/global/homes/s/shengyu/.conda/envs/rc_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedColumn.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [ ]:
REPEAT_DIR = f'/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1' 
d = Table.read(REPEAT_DIR+f'/QSOrepeats.fits', hdu=1)

In [ ]:
REPEAT_DIR = f'/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1' 
d = Table.read(REPEAT_DIR+f'/QSOrepeats.fits', hdu=1)
zmin = 0.8
zmax = 2.1
selz = ((zmin<d["Z_0"])&(d["Z_0"]<zmax))|((zmin<d["Z_1"])&(d["Z_1"]<zmax))
sel_catas = abs(d['DV'])>3000
d[selz & sel_catas]

TARGETID,TARGET_RA,TARGET_DEC,PHOTSYS,DESI_TARGET,BGS_TARGET,SURVEY_0,TILEID_0,LASTNIGHT_0,PETAL_LOC_0,FIBER_0,COADD_FIBERSTATUS_0,TSNR2_BGS_0,TSNR2_LRG_0,TSNR2_ELG_0,TSNR2_QSO_0,TSNR2_LYA_0,Z_0,ZERR_0,ZWARN_0,DELTACHI2_0,OII_FLUX_0,OII_FLUX_IVAR_0,Z_NEW_0,ZERR_NEW_0,IS_QSO_QN_NEW_RR_0,ZMTL_ZWARN_0,ZMTL_Z_QN_0,ZMTL_Z_QN_CONF_0,ZMTL_IS_QSO_QN_0,GOOD_BGS_0,GOOD_LRG_0,GOOD_ELG_0,GOOD_QSO_0,SURVEY_1,TILEID_1,LASTNIGHT_1,PETAL_LOC_1,FIBER_1,COADD_FIBERSTATUS_1,TSNR2_BGS_1,TSNR2_LRG_1,TSNR2_ELG_1,TSNR2_QSO_1,TSNR2_LYA_1,Z_1,ZERR_1,ZWARN_1,DELTACHI2_1,OII_FLUX_1,OII_FLUX_IVAR_1,Z_NEW_1,ZERR_NEW_1,IS_QSO_QN_NEW_RR_1,ZMTL_ZWARN_1,ZMTL_Z_QN_1,ZMTL_Z_QN_CONF_1,ZMTL_IS_QSO_QN_1,GOOD_BGS_1,GOOD_LRG_1,GOOD_ELG_1,GOOD_QSO_1,DV,DV_NEW
int64,float64,float64,bytes1,int64,int64,bytes4,int32,int32,int16,int32,int32,float32,float32,float32,float32,float32,float64,float64,int64,float64,float32,float32,float64,float32,bool,int64,float64,float64,int16,bool,bool,bool,bool,bytes4,int32,int32,int16,int32,int32,float32,float32,float32,float32,float32,float64,float64,int64,float64,float32,float32,float64,float32,bool,int64,float64,float64,int16,bool,bool,bool,bool,float64,float64
39627334763942114,62.41429578644092,-19.012558123878666,S,4611686018427650052,0,main,10243,20211103,0,159,0,6170.5703,73.281296,104.68083,28.031994,90.516266,1.6660638401922518,6.759132878018886e-05,0,19.176493525505066,--,--,2.5430464197795435,0.00046857048,True,0,2.5575442708917655,0.9999986109577833,1,False,False,False,True,main,7146,20221220,0,457,0,6175.2344,77.55812,113.8521,28.138481,87.41986,2.5430902450193686,0.000980711279919456,4,5.257853996939957,--,--,--,--,False,4,2.5471454579681563,0.999999266524066,1,False,False,False,True,98619.50703140875,--
39627339939711018,27.9983732539334,-18.722609206776017,S,4611686018427650052,0,main,9107,20211031,1,802,0,7532.526,89.49169,130.32314,35.282127,121.794106,1.625469532263648,0.0008135278351569729,0,347.97027962608263,--,--,--,--,False,0,1.6248397731449122,0.9999999977235363,1,True,False,False,True,main,4311,20231205,9,4927,0,7311.302,83.73307,120.10724,31.986294,179.83957,1.6190576468819584,0.000782222975742356,0,620.8077875552699,-0.5544501,2.0531964,--,--,False,0,1.619244355967,0.9999957583940197,1,True,False,False,True,-732.1489948252005,--
39627340275255870,49.08957020995101,-18.708253556750233,S,4611686018428305446,0,main,5578,20240115,1,876,0,6322.417,73.44809,102.90688,26.166744,86.528496,0.6726897212706846,8.827207359091725e-05,4,0.9271681755781174,2.3046036,0.5625291,1.8717108538833895,0.0009413722,True,4,1.891695280060401,0.9999996122682022,1,False,False,False,True,main,2446,20231207,9,4984,0,7533.015,89.10441,126.55614,34.16395,164.99431,1.908172028624551,0.0014300663679107635,0,38.71360712824389,--,--,--,--,False,0,1.8911494852920163,0.9999994441455812,1,False,False,False,True,221432.74573108266,--
39627340304620362,51.072010223479815,-18.73415535595483,S,4611686018428305446,0,main,5578,20240115,9,4516,0,6902.3906,76.99681,114.84182,28.513977,84.74939,2.1264313664842773,0.00040922789155693664,0,30.09002346545458,--,--,--,--,False,0,2.110210099059035,0.9999999985372952,1,False,False,False,True,main,2431,20211211,1,893,0,9923.741,119.62926,205.14374,45.321026,85.4882,2.085270154827261,0.0002846587915212237,0,33.26908776164055,--,--,--,--,False,0,2.1065035305106052,0.9999999994676816,1,False,False,False,True,-3946.9348181442565,--
39627340304620362,51.072010223479815,-18.73415535595483,S,4611686018428305446,0,main,5578,20240115,9,4516,0,6902.3906,76.99681,114.84182,28.513977,84.74939,2.1264313664842773,0.00040922789155693664,0,30.09002346545458,--,--,--,--,False,0,2.110210099059035,0.9999999985372952,1,False,False,False,True,main,10246,20240129,1,564,0,6740.451,81.354614,115.00851,31.25117,95.09421,2.0875203941710527,0.0003347790012987621,0,30.94591740705073,--,--,--,--,False,0,2.1081713631441534,0.9999999996100886,1,False,False,False,True,-3731.160120130599,--
39627340304620362,51.072010223479815,-18.73415535595483,

In [284]:
sel_catas = abs(d['DV'])>3000
d[selz & sel_catas]

TARGETID,TARGET_RA,TARGET_DEC,PHOTSYS,DESI_TARGET,BGS_TARGET,SURVEY_0,TILEID_0,LASTNIGHT_0,PETAL_LOC_0,FIBER_0,COADD_FIBERSTATUS_0,TSNR2_BGS_0,TSNR2_LRG_0,TSNR2_ELG_0,TSNR2_QSO_0,TSNR2_LYA_0,Z_0,ZERR_0,ZWARN_0,DELTACHI2_0,OII_FLUX_0,OII_FLUX_IVAR_0,Z_NEW_0,ZERR_NEW_0,IS_QSO_QN_NEW_RR_0,ZMTL_ZWARN_0,ZMTL_Z_QN_0,ZMTL_Z_QN_CONF_0,ZMTL_IS_QSO_QN_0,GOOD_BGS_0,GOOD_LRG_0,GOOD_ELG_0,GOOD_QSO_0,SURVEY_1,TILEID_1,LASTNIGHT_1,PETAL_LOC_1,FIBER_1,COADD_FIBERSTATUS_1,TSNR2_BGS_1,TSNR2_LRG_1,TSNR2_ELG_1,TSNR2_QSO_1,TSNR2_LYA_1,Z_1,ZERR_1,ZWARN_1,DELTACHI2_1,OII_FLUX_1,OII_FLUX_IVAR_1,Z_NEW_1,ZERR_NEW_1,IS_QSO_QN_NEW_RR_1,ZMTL_ZWARN_1,ZMTL_Z_QN_1,ZMTL_Z_QN_CONF_1,ZMTL_IS_QSO_QN_1,GOOD_BGS_1,GOOD_LRG_1,GOOD_ELG_1,GOOD_QSO_1,DV,DV_NEW
int64,float64,float64,bytes1,int64,int64,bytes4,int32,int32,int16,int32,int32,float32,float32,float32,float32,float32,float64,float64,int64,float64,float32,float32,float64,float32,bool,int64,float64,float64,int16,bool,bool,bool,bool,bytes4,int32,int32,int16,int32,int32,float32,float32,float32,float32,float32,float64,float64,int64,float64,float32,float32,float64,float32,bool,int64,float64,float64,int16,bool,bool,bool,bool,float64,float64
39627334763942114,62.41429578644092,-19.012558123878666,S,4611686018427650052,0,main,10243,20211103,0,159,0,6170.5703,73.281296,104.68083,28.031994,90.516266,1.6660638401922518,6.759132878018886e-05,0,19.176493525505066,--,--,2.5430464197795435,0.00046857048,True,0,2.5575442708917655,0.9999986109577833,1,False,False,False,True,main,7146,20221220,0,457,0,6175.2344,77.55812,113.8521,28.138481,87.41986,2.5430902450193686,0.000980711279919456,4,5.257853996939957,--,--,--,--,False,4,2.5471454579681563,0.999999266524066,1,False,False,False,True,98619.50703140875,--
39627340275255870,49.08957020995101,-18.708253556750233,S,4611686018428305446,0,main,5578,20240115,1,876,0,6322.417,73.44809,102.90688,26.166744,86.528496,0.6726897212706846,8.827207359091725e-05,4,0.9271681755781174,2.3046036,0.5625291,1.8717108538833895,0.0009413722,True,4,1.891695280060401,0.9999996122682022,1,False,False,False,True,main,2446,20231207,9,4984,0,7533.015,89.10441,126.55614,34.16395,164.99431,1.908172028624551,0.0014300663679107635,0,38.71360712824389,--,--,--,--,False,0,1.8911494852920163,0.9999994441455812,1,False,False,False,True,221432.74573108266,--
39627340304620362,51.072010223479815,-18.73415535595483,S,4611686018428305446,0,main,5578,20240115,9,4516,0,6902.3906,76.99681,114.84182,28.513977,84.74939,2.1264313664842773,0.00040922789155693664,0,30.09002346545458,--,--,--,--,False,0,2.110210099059035,0.9999999985372952,1,False,False,False,True,main,2431,20211211,1,893,0,9923.741,119.62926,205.14374,45.321026,85.4882,2.085270154827261,0.0002846587915212237,0,33.26908776164055,--,--,--,--,False,0,2.1065035305106052,0.9999999994676816,1,False,False,False,True,-3946.9348181442565,--
39627340304620362,51.072010223479815,-18.73415535595483,S,4611686018428305446,0,main,5578,20240115,9,4516,0,6902.3906,76.99681,114.84182,28.513977,84.74939,2.1264313664842773,0.00040922789155693664,0,30.09002346545458,--,--,--,--,False,0,2.110210099059035,0.9999999985372952,1,False,False,False,True,main,10246,20240129,1,564,0,6740.451,81.354614,115.00851,31.25117,95.09421,2.0875203941710527,0.0003347790012987621,0,30.94591740705073,--,--,--,--,False,0,2.1081713631441534,0.9999999996100886,1,False,False,False,True,-3731.160120130599,--
39627340308809958,51.187873749296216,-18.68498663548925,S,4611686018427650052,0,main,2431,20211211,1,802,0,9178.343,109.40445,187.0044,40.241814,74.7603,2.0069728172974117,0.0007477506401380655,0,48.49606710206717,--,--,--,--,False,0,1.9983404760103864,0.9999997717231851,1,True,False,False,True,main,5578,20240115,9,4821,0,7212.379,85.19447,125.80625,31.0412,101.923996,0.7043309361524931,0.00024905586442791557,0,12.884191201068461,1.5745574,0.9091406,2.011520769106854,0.0006324647,True,0,2.0054840193132675,0.9997103015338709,1,False,False,False,True,-129872.21207844844,--
39627345711077425,30.712942

## Consistent check

In [18]:
tracer = 'LRG'
zmin = 0.4
zmax = 0.6
repeat_fn = f'/global/cfs/cdirs/desi/survey/catalogs/DA2/LSS/loa-v1/{tracer}repeats.fits'
d = Table.read(repeat_fn)
sel = np.isfinite(d['Z1']) & np.isfinite(d['Z2'])
# selz = ((zmin<d["Z1"])&(d["Z1"]<zmax))|((zmin<d["Z2"])&(d["Z2"]<zmax))
# d = d[sel & selz]
d = d[sel]

dz = abs(d['Z1']-d['Z2'])/(1+d['Z1'])
print(f'{len(dz[dz>0.0033])/len(dz)*100:.3f}%')

0.602%


## Do the selection for the repeats-dark-pairs.fits from Anand script

we don't do the z selection here

In [ ]:
from desitarget.targetmask import desi_mask, bgs_mask
from desitarget.sv3.sv3_targetmask import desi_mask as sv3_desi_mask
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask
from desitarget.targetmask import zwarn_mask as zmtl_zwarn_mask

#generate the clean repeats from Anand scripts
REPEAT_AN_DIR = f'/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1' 

tracer = 'QSO' # BGS_BRIGHT, LRG, ELG, QSO, BGS_FAINT?
prog = 'dark' # dark, bright

if prog == "bright":
    tracers = ["BGS_BRIGHT", "BGS_FAINT"]
    repeat_fn = REPEAT_AN_DIR+ f'/repeats-bright-pairs.fits'
    mask, mask_key = bgs_mask, "BGS_TARGET"
    effkey, effmin, effmax, effxlim = "TSNR2_BGS", 0.85 * 180, 1.5 * 180, (0, 500)
if prog == "dark":
    tracers = ["LRG", "ELG_LOP", "ELG_VLO"]
    repeat_fn = REPEAT_AN_DIR+ f'/repeats-dark-pairs.fits'
    mask, mask_key = desi_mask, "DESI_TARGET"
    effkey, effmin, effmax, effxlim = (
        "TSNR2_LRG",
        0.85 * 1000,
        1.5 * 1000,
        (500, 1500),
    )
repeat_fn
d = Table.read(repeat_fn, hdu=1)

# AR per-tracer setting
if tracer[:3] == "BGS":
    zmin, zmax, goodkey = 0.1, 0.4, "GOOD_BGS"
else:
    if tracer == "LRG":
        zmin, zmax, goodkey = 0.4, 1.1, "GOOD_LRG"
    elif tracer[:3] == 'ELG':
        zmin, zmax, goodkey = 0.6, 1.6, "GOOD_ELG"
    elif tracer[:3] == 'QSO':
        zmin, zmax, goodkey = 0.8, 2.1, "GOOD_QSO"
        
# AR
if tracer[:3] == "ELG":
    xs = np.log10(
        d["OII_FLUX_0"] * np.sqrt(d["OII_FLUX_IVAR_0"])
    ) + 0.2 * np.log10(d["DELTACHI2_0"])
    xlim = (0, 3)
    xlab = "log10(FOII_SNR) + 0.2 * log10(DELTACHI2)"
else:
    xs = np.log10(d["DELTACHI2_0"])
    xlab = "log10(DELTACHI2)"
    xlim = (0, 6)

# AR efftime_spec
snr2time = d.meta["{}SNR2T".format(effkey.split("_")[1])]
efftime0s = snr2time * d["{}_0".format(effkey)]
efftime1s = snr2time * d["{}_1".format(effkey)]
# AR for both elements of a pair, cut on:
# AR - tracer
# AR - coadd_fiberstatus
# AR - zmtl_zwarn_mask nodata + bad
# AR - efftime_spec
# AR cut on redshift range with a OR
sel = (d[mask_key] & mask[tracer]) > 0
sel &= (d["COADD_FIBERSTATUS_0"] == 0) & (d["COADD_FIBERSTATUS_1"] == 0)
nodata0 = (d["ZMTL_ZWARN_0"] & zmtl_zwarn_mask["NODATA"]) > 0
nodata1 = (d["ZMTL_ZWARN_1"] & zmtl_zwarn_mask["NODATA"]) > 0
badqa0 = (d["ZMTL_ZWARN_0"] & zmtl_zwarn_mask.mask("BAD_SPECQA|BAD_PETALQA")) > 0
badqa1 = (d["ZMTL_ZWARN_1"] & zmtl_zwarn_mask.mask("BAD_SPECQA|BAD_PETALQA")) > 0
sel &= (~nodata0) & (~nodata1)
sel &= (~badqa0) & (~badqa1)
sel &= (efftime0s > effmin) & (efftime1s > effmin)
sel &= (efftime0s < effmax) & (efftime1s < effmax)
sel &= (d["{}_0".format(goodkey)]) & (d["{}_1".format(goodkey)])
# sel &= (d['SURVEY_0']=='sv3')&(d['SURVEY_1']=='sv3')
# sel &= (d['SURVEY_0']=='main')&(d['SURVEY_1']=='main')
# selz0 = (d["Z_0"] > zmin) & (d["Z_0"] < zmax)
# selz1 = (d["Z_1"] > zmin) & (d["Z_1"] < zmax)
# sel &= (selz0) | (selz1)
if tracer == 'ELG':
    sel &= (~d["GOOD_QSO_0"]) & (~d["GOOD_QSO_1"])
if tracer == 'QSO':
    sel &= (~d["GOOD_ELG_0"]) & (~d["GOOD_ELG_0"])
    sel &= (~d["GOOD_LRG_0"]) & (~d["GOOD_LRG_0"])
d= d[~np.isnan(d['DV'])]
d = d[sel]
print(d.meta)
outfn = f"/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1/{tracer}repeats.fits"
# write the table; this preserves d.meta (use serialize_meta=True for full fidelity)
d.write(outfn, format="fits", overwrite=True)
d

OrderedDict([('PRODDIR', '/dvs_ro/cfs/cdirs/desi/spectro/redux/kibo'), ('ZCATDIR', '/dvs_ro/cfs/cdirs/desi/spectro/redux/kibo/zcatalog/v1'), ('PROG', 'dark'), ('BGSSNR2T', 0.1400000003119347), ('LRGSNR2T', 12.15060572324326), ('ELGSNR2T', 8.60000036161418), ('QSOSNR2T', 33.6094737399845), ('LYASNR2T', 11.80090901380597)])


TARGETID,TARGET_RA,TARGET_DEC,PHOTSYS,DESI_TARGET,BGS_TARGET,SURVEY_0,TILEID_0,LASTNIGHT_0,PETAL_LOC_0,FIBER_0,COADD_FIBERSTATUS_0,TSNR2_BGS_0,TSNR2_LRG_0,TSNR2_ELG_0,TSNR2_QSO_0,TSNR2_LYA_0,Z_0,ZERR_0,ZWARN_0,DELTACHI2_0,OII_FLUX_0,OII_FLUX_IVAR_0,Z_NEW_0,ZERR_NEW_0,IS_QSO_QN_NEW_RR_0,ZMTL_ZWARN_0,ZMTL_Z_QN_0,ZMTL_Z_QN_CONF_0,ZMTL_IS_QSO_QN_0,GOOD_BGS_0,GOOD_LRG_0,GOOD_ELG_0,GOOD_QSO_0,SURVEY_1,TILEID_1,LASTNIGHT_1,PETAL_LOC_1,FIBER_1,COADD_FIBERSTATUS_1,TSNR2_BGS_1,TSNR2_LRG_1,TSNR2_ELG_1,TSNR2_QSO_1,TSNR2_LYA_1,Z_1,ZERR_1,ZWARN_1,DELTACHI2_1,OII_FLUX_1,OII_FLUX_IVAR_1,Z_NEW_1,ZERR_NEW_1,IS_QSO_QN_NEW_RR_1,ZMTL_ZWARN_1,ZMTL_Z_QN_1,ZMTL_Z_QN_CONF_1,ZMTL_IS_QSO_QN_1,GOOD_BGS_1,GOOD_LRG_1,GOOD_ELG_1,GOOD_QSO_1,DV,DV_NEW
int64,float64,float64,bytes1,int64,int64,bytes4,int32,int32,int16,int32,int32,float32,float32,float32,float32,float32,float64,float64,int64,float64,float32,float32,float64,float32,bool,int64,float64,float64,int16,bool,bool,bool,bool,bytes4,int32,int32,int16,int32,int32,float32,float32,float32,float32,float32,float64,float64,int64,float64,float32,float32,float64,float32,bool,int64,float64,float64,int16,bool,bool,bool,bool,float64,float64
39627328883524109,51.8199242259905,-19.263063749506014,S,4611686018428305446,0,main,10246,20240129,0,336,0,6763.1553,82.52919,115.37672,30.360767,104.57906,2.6311708603459234,0.0001498512495028387,0,12956.742910504341,--,--,--,--,False,0,2.632636263423785,0.9999999854686108,1,True,False,False,True,main,2431,20211211,0,478,0,9188.407,111.12407,182.87225,40.437347,78.1615,2.630022854814536,0.00013406065703960195,0,13106.382732197642,--,--,--,--,False,0,2.638018524840899,0.9999999823858786,1,True,False,False,True,-94.78028252833774,--
39627328883532704,52.07007140011368,-19.133856644131157,S,262148,0,main,2431,20211211,0,195,0,8831.949,105.309074,171.79227,37.750027,65.21117,3.2862124471404863,0.0003236922383525863,0,786.4917979062302,--,--,--,--,False,0,3.296686558262974,0.9999999010751297,1,True,False,False,True,main,10246,20240129,0,305,0,7002.581,85.98608,121.16713,31.818363,109.791664,3.2856237982576753,0.0003724867341892688,0,839.3955156467855,--,--,--,--,False,0,3.2821819647402632,0.9999999945801739,1,True,False,False,True,-41.17212985898944,--
39627329047102947,62.150229048163645,-19.165454687808406,S,4611686018427650052,0,main,7146,20221220,0,173,0,5635.6157,70.47236,103.476456,25.086185,50.998543,2.6666787346704264,0.0002508721479947322,0,4921.510086748749,--,--,--,--,False,0,2.668320532784264,0.9999997704104325,1,True,False,False,True,main,10243,20211103,0,495,0,6245.7393,74.981445,106.619316,28.771717,123.2391,2.666921716784773,0.0002291651649820853,0,5138.65089841187,--,--,--,--,False,0,2.6699752834498596,0.999998862422226,1,True,False,False,True,19.86653606199166,--
39627334583584355,50.97773346766405,-18.893384995196307,S,917542,0,main,2431,20211211,1,699,0,9145.879,108.37106,175.15996,39.346485,55.053177,2.4636605860314074,0.0005445591161995427,0,88.22283034026623,--,--,--,--,False,0,2.4598590198154597,0.9999997808984153,1,True,False,False,True,main,5578,20240115,9,4942,0,7191.9927,84.01433,124.2234,30.621271,100.35766,2.4639688646107216,0.0005049385462730434,0,405.2141877859831,--,--,--,--,False,0,2.4552721158061326,0.9999997369335324,1,True,False,False,True,26.68263553712172,--
39627334583587384,51.05614440905087,-18.90874468514607,S,4611686018428305446,0,main,2431,20211211,1,689,0,9359.968,110.091805,177.60623,39.761303,58.484077,2.3541358883958226,0.0011022302352320318,0,169.23111809045076,--,--,--,--,False,0,2.353965540218096,0.9999998979070541,1,True,False,False,True,main,10246,20240129,1,675,0,6817.706,82.0188,113.98394,31.194584,94.97974,2.356030988531319,0.0005088709577531282,0,335.6650383503875,--,--,--,--,False,0,2.3530009881329677,0.9999991675817169,1,True,False,False,True,169.3839327566311,--
39627334587784469,51.41541178842179,-19.00810952353072,S,917542,0,main,10246,20240129,0,53,0,6033.8945,71.3961,99.87338,26.701622,71.87366,2.490077386768722,0.00113

In [ ]:
from desitarget.targetmask import desi_mask, bgs_mask
from desitarget.sv3.sv3_targetmask import desi_mask as sv3_desi_mask
from desitarget.sv3.sv3_targetmask import bgs_mask as sv3_bgs_mask
from desitarget.targetmask import zwarn_mask as zmtl_zwarn_mask
from astropy import constants

tracer = 'QSO'
REPEAT_AN_DIR = f'/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1' 
repeat_fn = REPEAT_AN_DIR+ f'/repeats-dark-pairs.fits'
outfn = f"/pscratch/sd/s/shengyu/repeats/DA2/kibo-v1/QSOrepeats.fits"

d        = Table.read(repeat_fn)
tracer   = 'QSO'
goodkey  = "GOOD_QSO"
mask, mask_key = desi_mask, "DESI_TARGET"
effkey, effmin, effmax, effxlim = ("TSNR2_LRG",0.85 * 1000,1.5 * 1000,(500, 1500))
# efftime_spec calculation for selections
snr2time = d.meta["{}SNR2T".format(effkey.split("_")[1])]
efftime0s= snr2time * d["{}_0".format(effkey)]
efftime1s= snr2time * d["{}_1".format(effkey)]
# zmtl_zwarn_mask nodata + bad selections
nodata0  = (d["ZMTL_ZWARN_0"] & zmtl_zwarn_mask["NODATA"]) > 0
nodata1  = (d["ZMTL_ZWARN_1"] & zmtl_zwarn_mask["NODATA"]) > 0
badqa0   = (d["ZMTL_ZWARN_0"] & zmtl_zwarn_mask.mask("BAD_SPECQA|BAD_PETALQA")) > 0
badqa1   = (d["ZMTL_ZWARN_1"] & zmtl_zwarn_mask.mask("BAD_SPECQA|BAD_PETALQA")) > 0
# Apply the selection criteria to clean the data
sel      = (d[mask_key] & mask[tracer]) > 0
sel &= (d["COADD_FIBERSTATUS_0"] == 0) & (d["COADD_FIBERSTATUS_1"] == 0)
sel &= (~nodata0) & (~nodata1)
sel &= (~badqa0) & (~badqa1)
sel &= (efftime0s > effmin) & (efftime1s > effmin)
sel &= (efftime0s < effmax) & (efftime1s < effmax)
sel &= (d["{}_0".format(goodkey)]) & (d["{}_1".format(goodkey)])
sel &= (d['SURVEY_0']=='main')&(d['SURVEY_1']=='main')
print('write QSO-only file')
d    = d[sel]
d.write(outfn,overwrite=True)

if np.isnan(d["DV"]).sum() == 0:
    print("QSO redshifts needs to be corrected")
    columns  = ['TARGETID','TILEID','LASTNIGHT','Z']
    d_zacc   = Table(fitsio.read('/global/cfs/cdirs/desi/survey/catalogs/DA2/QSO/kibo/QSO_cat_kibo_cumulative_v3.fits',columns=columns))
    # change the name in the data file to be match with the accurate redshift table
    for rid in range(2):
        for col in columns[1:]:
            if col not in d.colnames:
                d[f'{col}_{rid}'].name = col
        # join the two tables to obtain the true QSO measurement from d_zacc in d_ztrue
        d_ztrue  = []
        d_ztrue  = join(d,d_zacc,keys=columns[:-1],join_type='left',table_names=[f'{rid}', f'{rid}_true'])
        d_ztrue[f'Z_{rid}'] = d_ztrue[f'Z_{rid}_true']*1
        # return to the *_{rid} names
        for col in columns[1:-1]:
            d_ztrue[f'{col}'].name = f'{col}_{rid}'
        # 
        d = d_ztrue.copy()
    # save the new file
    d_ztrue['DV'].name = 'DV_Redrock'
    d_ztrue['DV'] = constants.c.to("km/s").value * (d_ztrue["Z_1"] - d_ztrue["Z_0"]) / (1 + d_ztrue["Z_0"])
    d_ztrue['Z_0'][d_ztrue.mask['Z_0_true']] = np.nan
    d_ztrue['Z_1'][d_ztrue.mask['Z_1_true']] = np.nan
    d_ztrue['DV'][d_ztrue.mask['DV']]= np.nan #Redrock redshift
    d_ztrue.remove_columns(['Z_0_true','Z_1_true'])
    d_ztrue.write(outfn,overwrite=True,format='fits')

write QSO-only file
QSO redshifts needs to be corrected
